In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd

# Importing Data. Familiarization

In [3]:
traindf = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
testdf = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
traindf.shape

(1460, 81)

In [5]:
traindf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
traindf.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# Data Preprocessing

## Dropping Columns with majority null information

In [7]:
traindf=traindf.drop(['Alley','MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'], axis=1)
testdf=testdf.drop(['Alley','MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'], axis=1)
traindf_ids = traindf['Id']
testdf_ids = testdf['Id']
traindf = traindf.drop('Id',axis=1)
testdf = testdf.drop('Id',axis=1)

## Replacing NAN in categorical features
The object of the following code is to replace the NAN values in the dtype object attributes with their most frequently occurring element.

Though we could use to gain the features of dtype object, we also want to gather their associated values:

object_train_columns = traindf.select_dtypes(include=object).columns
object_test_columns = testdf.select_dtypes(include=object).columns



In [8]:
object_cols_with_nan_train = {}
for i, val in enumerate(traindf.columns):
    if traindf[val].dtypes == object:
        if traindf[val].isnull().sum() > 0:
            print(i, val, traindf[val].unique())
            object_cols_with_nan_train[val] = traindf[val].unique()

27 BsmtQual ['Gd' 'TA' 'Ex' nan 'Fa']
28 BsmtCond ['TA' 'Gd' nan 'Fa' 'Po']
29 BsmtExposure ['No' 'Gd' 'Mn' 'Av' nan]
30 BsmtFinType1 ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
32 BsmtFinType2 ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
39 Electrical ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
54 GarageType ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
56 GarageFinish ['RFn' 'Unf' 'Fin' nan]
59 GarageQual ['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
60 GarageCond ['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [9]:
object_cols_with_nan_test = {}
for i, val in enumerate(testdf.columns):
    if testdf[val].dtypes == object:
        if testdf[val].isnull().sum() > 0:
            print(i, val, testdf[val].unique())
            object_cols_with_nan_test[val] = testdf[val].unique()

1 MSZoning ['RH' 'RL' 'RM' 'FV' 'C (all)' nan]
7 Utilities ['AllPub' nan]
21 Exterior1st ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'CemntBd' 'WdShing'
 'BrkFace' 'AsbShng' 'BrkComm' 'Stucco' 'AsphShn' nan 'CBlock']
22 Exterior2nd ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'Brk Cmn' 'CmentBd'
 'ImStucc' 'Wd Shng' 'AsbShng' 'Stucco' 'CBlock' 'BrkFace' 'AsphShn' nan
 'Stone']
27 BsmtQual ['TA' 'Gd' 'Ex' 'Fa' nan]
28 BsmtCond ['TA' 'Po' 'Fa' 'Gd' nan]
29 BsmtExposure ['No' 'Gd' 'Mn' 'Av' nan]
30 BsmtFinType1 ['Rec' 'ALQ' 'GLQ' 'Unf' 'BLQ' 'LwQ' nan]
32 BsmtFinType2 ['LwQ' 'Unf' 'Rec' 'BLQ' 'GLQ' 'ALQ' nan]
50 KitchenQual ['TA' 'Gd' 'Ex' 'Fa' nan]
52 Functional ['Typ' 'Min2' 'Min1' 'Mod' 'Maj1' 'Sev' 'Maj2' nan]
54 GarageType ['Attchd' 'Detchd' 'BuiltIn' nan 'Basment' '2Types' 'CarPort']
56 GarageFinish ['Unf' 'Fin' 'RFn' nan]
59 GarageQual ['TA' nan 'Fa' 'Gd' 'Po']
60 GarageCond ['TA' nan 'Fa' 'Gd' 'Po' 'Ex']
71 SaleType ['WD' 'COD' 'New' 'ConLD' 'Oth' 'Con' 'ConLw' 'ConLI

In [10]:
col_value_counts_train = {}
for col in object_cols_with_nan_train.keys():
    temp = traindf[col].value_counts(normalize=True)
    col_value_counts_train[col] = temp

In [11]:
col_value_counts_test = {}
for col in object_cols_with_nan_test.keys():
    temp = testdf[col].value_counts(normalize=True)
    col_value_counts_test[col] = temp

In [12]:
out_key_train = []
max_key_train = []
key_values_train = []
for outer_dict_key, inner_dict_keys in col_value_counts_train.items():
    highest_val = 0
    for k in inner_dict_keys.keys():
        #print(outer_dict_key,'(', k,'): ',col_value_counts[outer_dict_key][k])
        if highest_val < col_value_counts_train[outer_dict_key][k]:
            highest_val = col_value_counts_train[outer_dict_key][k]
    out_key_train.append(outer_dict_key)
    max_key_train.append(k)
    key_values_train.append(highest_val)
        

In [13]:
out_key_test = []
max_key_test = []
key_values_test = []
for outer_dict_key, inner_dict_keys in col_value_counts_test.items():
    highest_val = 0
    for k in inner_dict_keys.keys():
        #print(outer_dict_key,'(', k,'): ',col_value_counts[outer_dict_key][k])
        if highest_val < col_value_counts_test[outer_dict_key][k]:
            highest_val = col_value_counts_test[outer_dict_key][k]
    out_key_test.append(outer_dict_key)
    max_key_test.append(k)
    key_values_test.append(highest_val)

In [14]:
nan_replace_train = list(zip(out_key_train, max_key_train, key_values_train))

In [15]:
nan_replace_test = list(zip(out_key_test, max_key_test, key_values_test))

In [16]:
#fill NAN values in object columns with most occurring value
for tup in nan_replace_train:
    traindf[tup[0]].fillna(tup[1], inplace=True)

/tmp/ipykernel_395/938530933.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  traindf[tup[0]].fillna(tup[1], inplace=True)


In [17]:
for tup in nan_replace_test:
    testdf[tup[0]].fillna(tup[1], inplace=True)

/tmp/ipykernel_395/1031643080.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  testdf[tup[0]].fillna(tup[1], inplace=True)


## Replacing NAN attributes in features with dtype of int or float
The goal of the following code is to replace NULL values with the average of the rest of the attributes in the column

In [18]:
num_cols_with_nan_train = traindf.select_dtypes(include=[int, float]).columns
num_cols_with_nan_test = testdf.select_dtypes(include=[int,float]).columns

In [19]:
#using average to fill NAN values in numeric columns
for col in num_cols_with_nan_train:
    traindf.fillna({col: traindf[col].mean()}, inplace=True)

In [20]:
for col in num_cols_with_nan_test:
    testdf.fillna({col: testdf[col].mean()}, inplace=True)

No more null values in traindf

# Categorical Encoding
The goal of the following code is to encode the categorical information for the train data and the test data with the same features and encoding

In [21]:
object_train_columns = traindf.select_dtypes(include=object).columns
object_test_columns = testdf.select_dtypes(include=object).columns

In [22]:
traindf_targets = traindf['SalePrice']
traindf = traindf.drop('SalePrice', axis=1)

In [23]:
#!pip install category_encoders

We need to do some feature engineering

In [24]:
from category_encoders import *

In [25]:
enc = BinaryEncoder(cols=object_train_columns).fit(traindf)

In [26]:
traindf = enc.transform(traindf)

In [27]:
testdf = enc.transform(testdf)

In [28]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [29]:
traindf.head(10)

,MSSubClass,MSZoning_0,MSZoning_1,MSZoning_2,LotFrontage,LotArea,Street_0,Street_1,LotShape_0,LotShape_1,LotShape_2,LandContour_0,LandContour_1,LandContour_2,Utilities_0,Utilities_1,LotConfig_0,LotConfig_1,LotConfig_2,LandSlope_0,LandSlope_1,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Condition1_0,Condition1_1,Condition1_2,Condition1_3,Condition2_0,Condition2_1,Condition2_2,Condition2_3,BldgType_0,BldgType_1,BldgType_2,HouseStyle_0,HouseStyle_1,HouseStyle_2,HouseStyle_3,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle_0,RoofStyle_1,RoofStyle_2,RoofMatl_0,RoofMatl_1,RoofMatl_2,RoofMatl_3,Exterior1st_0,Exterior1st_1,Exterior1st_2,Exterior1st_3,Exterior2nd_0,Exterior2nd_1,Exterior2nd_2,Exterior2nd_3,Exterior2nd_4,MasVnrArea,ExterQual_0,ExterQual_1,ExterQual_2,ExterCond_0,ExterCond_1,ExterCond_2,Foundation_0,Foundation_1,Foundation_2,BsmtQual_0,BsmtQual_1,BsmtQual_2,BsmtCond_0,BsmtCond_1,BsmtCond_2,BsmtExposure_0,BsmtExposure_1,BsmtExposure_2,BsmtFinType1_0,BsmtFinType1_1,BsmtFinType1_2,BsmtFinSF1,BsmtFinType2_0,BsmtFinType2_1,BsmtFinType2_2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating_0,Heating_1,Heating_2,HeatingQC_0,HeatingQC_1,HeatingQC_2,CentralAir_0,CentralAir_1,Electrical_0,Electrical_1,Electrical_2,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual_0,KitchenQual_1,KitchenQual_2,TotRmsAbvGrd,Functional_0,Functional_1,Functional_2,Fireplaces,GarageType_0,GarageType_1,GarageType_2,GarageYrBlt,GarageFinish_0,GarageFinish_1,GarageCars,GarageArea,GarageQual_0,GarageQual_1,GarageQual_2,GarageCond_0,GarageCond_1,GarageCond_2,PavedDrive_0,PavedDrive_1,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType_0,SaleType_1,SaleType_2,SaleType_3,SaleCondition_0,SaleCondition_1,SaleCondition_2
0,60,0,0,1,65.000000,8450,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,7,5,2003,2003,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,196.0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,706,0,0,1,0,150,856,0,0,1,0,0,1,0,1,0,0,1,856,854,0,1710,1,0,2,1,3,1,0,0,1,8,0,0,1,0,0,0,1,2003.0,0,1,2,548,0,0,1,0,0,1,0,1,0,61,0,0,0,0,0,2,2008,0,0,0,1,0,0,1
1,20,0,0,1,80.000000,9600,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,6,8,1976,1976,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0.0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,978,0,0,1,0,284,1262,0,0,1,0,0,1,0,1,0,0,1,1262,0,0,1262,0,1,2,0,3,1,0,1,0,6,0,0,1,1,0,0,1,1976.0,0,1,2,460,0,0,1,0,0,1,0,1,298,0,0,0,0,0,0,5,2007,0,0,0,1,0,0,1
2,60,0,0,1,68.000000,11250,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,7,5,2001,2002,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,162.0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,1,486,0,0,1,0,434,920,0,0,1,0,0,1,0,1,0,0,1,920,866,0,1786,1,0,2,1,3,1,0,0,1,6,0,0,1,1,0,0,1,2001.0,0,1,2,608,0,0,1,0,0,1,0,1,0,42,0,0,0,0,0,9,2008,0,0,0,1,0,0,1
3,70,0,0,1,60.000000,9550,0,1,0,1,0,0,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,7,5,1915,1970,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0.0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,216,0,0,1,0,540,756,0,0,1,0,1,0,0,1,0,0,1,961,756,0,1717,1,0,1,0,3,1,0,0,1,7,0,0,1,1,0,1,0,1998.0,1,0,3,642,0,0,1,0,0,1,0,1,0,35,272,0,0,0,0,2,2006,0,0,0,1,0,1,0
4,60,0,0,1,84.000000,14260,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,8,5,2000,2000,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,350.0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,655,0,0,1,0,490,1145,0,0,1,0,0,1,0,1,0,0,1,1145,1053,0,2198,1,0,2,1,4,1,0,0,1,9,0,0,1,1,0,0,1,2000.0,0,1,3,836,0,0,1,0,0,1,0,1,192,84,0,0,0,0,0,12,2008,0,0,0,1,0,0,1
5,50,0,0,1,85.000000,14115,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,5,5,1993,1995,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0.0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,732,0,0,1,0,64,796,0,0,1,0,0,1,0,1,0,0,1,796,566,0,1362,1,0,1,1,1,1,0,1,0,5,0,0,1,0,0,0,1,1993.0,1,0,2,480,0,0,1,0,0,1,0,1,40,30,0,320,0,0,700,10,2009,0,0,0,1,0,0,1
6,20,0,0,1,75.000000,10084,

### Taking most correlated features only

In [30]:
corr = traindf.corrwith(traindf_targets).sort_values(ascending=False)

In [32]:
traindf_feature_exclusions = []
for k,v in corr.items():
    if v < 0.4:
        traindf_feature_exclusions.append(k)

In [33]:
traindf = traindf.drop(traindf_feature_exclusions, axis=1)

# Data Visualization

In [34]:
import seaborn as sns
import matplotlib.pyplot as plt

# Training Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(traindf, traindf_targets, test_size=0.2, random_state=1)

In [ ]:
model = Pipeline([('scaler', StandardScaler()),('rf', RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 14, random_state = 1))])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

In [ ]:
print("MSE ", mean_squared_error(y_test, yhat))
print("R2 ", r2_score(y_test, yhat))

In [ ]:
final = model.predict(testdf)

In [ ]:
submission = {'Id': testdf_ids, 'SalePrice': final}

In [ ]:
submission_df = pd.DataFrame.from_dict(submission)

In [ ]:
submission_df.to_csv('submission.csv', index=False)